In [2]:
import requests
import csv
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
import os
from numpy import arange

In [12]:
inexp_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [13]:
mod_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.2&ed_attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2'

***Pages of reviews for each resturant*** <br>
1) We can use all characters between "datePublished.....and... /iv>" to capture the actual texts of reviews <br>
2) We can also use "ratingValue" content=" as a marker for star rating <br>
3) we will have to use the total number of reviews (found in plain html on the first page of all resturants) to calculate how many pages of reviews (assuming 20 reviews per page) 

***Pages of resturants for each price point***<br>
1) We can calculate total pages by using the total number of resturants (found in plain html at the top of each price point page) and then we assume 30 resturants per page <br>
2) We will have to also scrape all resturant names first and convert to all small case with dashes put between each word and punctuation marks removed in order to construct URLs for individual resturants for pre-review scraping <br>
3) We can use "ranking"....."name"......"neighbohoods" as markers for each resturant name we have to scrape and process

***Scrapper Pseudo Code***<br>
1- Calculate the number of pages of resturants each price point has <br>
2- Extract and process all resturant names for each price point <br>
3- Calculate the number of pages of reviews each resturant has <br>
4- Use numbers from above to establish dataframes and loops <br>
5- Finally scrape all reviews and star ratings into 4 seperate dataframes, one for each price point </br>

In [73]:
#Put price point pages in list
price_points = list()
price_points.append(inexp_page1)
price_points.append(mod_page1)
price_points.append(pricey_page1)

In [74]:
#list for resturants in each price point
pp_rest = list()

In [75]:
#get number of pages of resturants for each price point
for price_point in price_points:
    response = requests.get(price_point)
    content = response.content
    pp_rest.append(re.findall('Showing 1-30 of....',str(content)))

In [76]:
pp_rest

[['Showing 1-30 of 447'], ['Showing 1-30 of 819'], ['Showing 1-30 of 107']]

In [77]:
#dictionary for pages
price_rests = dict()
#extract last 3 numbers
pp_rest[0] = str(pp_pages[0])
rests = re.findall('[0-9]{3}',str(pp_pages[0]))
rests = str(rests)
rests = rests[2:5]
price_rests['inexp'] = int(rests)
#---------------------------------------
pp_rest[1] = str(pp_pages[1])
rests = re.findall('[0-9]{3}',str(pp_pages[1]))
rests = str(rests)
rests = rests[2:5]
price_rests['mod'] = int(rests)
#---------------------------------------
pp_rest[2] = str(pp_pages[2])
rests = re.findall('[0-9]{3}',str(pp_pages[2]))
rests = str(rests)
rests = rests[2:5]
price_rests['pricey'] = int(rests)

In [79]:
price_rests

{'inexp': 447, 'mod': 819, 'pricey': 107}

In [86]:
#calculate number of pages using number of resturants
pp_pages = dict()
for pp in price_rests:
    pages = (price_rests[pp]/30) + 1
    pp_pages[pp] = int(pages)

In [87]:
pp_pages

{'inexp': 15, 'mod': 28, 'pricey': 4}